In [1]:
import warnings
from pathlib import Path
import pandas as pd
import re

In [2]:
# File Path
sdoh_measures_data_to_load = Path("../Data/SDOH_Measures_for_County__ACS_2017-2021_20240909.csv")

# Read shcool metrics into Pandas DataFrames
sdoh_measures_df = pd.read_csv(sdoh_measures_data_to_load)

sdoh_measures_df.head()

,Year,StateAbbr,StateDesc,LocationName,DataSource,Category,Measure,Data_Value_Unit,Data_Value_Type,Data_Value,MOE,TotalPopulation,LocationID,CategoryID,MeasureID,DataValueTypeID,Short_Question_Text,Geolocation
0,2017-2021,AL,Alabama,Coosa County,5-year ACS,SDOH,Housing cost burden among households,%,Percentage,12.8,4.0,10442,1037,SDOH,HCOST,Percent,Housing cost burden,POINT (-86.2434818 32.9314453)
1,2017-2021,AL,Alabama,Lamar County,5-year ACS,SDOH,Persons living below 150% of the poverty level,%,Percentage,27.4,4.0,13929,1075,SDOH,POV150,Percent,Poverty,POINT (-88.0874309 33.7870852)
2,2017-2021,AK,Alaska,Aleutians East Borough,5-year ACS,SDOH,Crowding among housing units,%,Percentage,4.6,2.3,3409,2013,SDOH,CROWD,Percent,Crowding,POINT (-161.9974772 55.2450437)
3,2017-2021,AR,Arkansas,Dallas County,5-year ACS,SDOH,Crowding among housing units,%,Percentage,2.3,2.2,6645,5039,SDOH,CROWD,Percent,Crowding,POINT (-92.6539989 33.9678233)
4,2017-2021,AR,Arkansas,Franklin County,5-year ACS,SDOH,Crowding among housing units,%,Percentage,2.6,1.8,17159,5047,SDOH,CROWD,Percent,Crowding,POINT (-93.8876647 35.5085579)


In [3]:
# Drop Unnecessary Columns
sdoh_measures_df.drop(columns=['Year',
                               'DataSource', 
                               'Category', 
                               'Data_Value_Unit',
                               'Data_Value_Type', 
                               'CategoryID', 
                               'DataValueTypeID'
                              ], 
                      inplace=True)

In [4]:
# Rename Columns
sdoh_measures_df.rename(columns={'StateAbbr': 'State_Abbr', 
                                 'StateDesc': 'State_Desc', 
                                 'LocationName': 'County', 
                                 'Data_Value': 'Pct_of_Pop', 
                                 'TotalPopulation': 'Total_Population', 
                                 'LocationID': 'CountyFIPS',
                                 'MeasureID': 'Measure_ID', 
                                 'Short_Question_Text': 'Measure_Short_Name'
                                }, 
                        inplace=True)

In [5]:
# Replace missing values in the 'County' column with a United States
# sdoh_measures_df['County'].fillna('United States', inplace=True)

In [6]:
# Function to extract latitude and longitude
def extract_lat_lng(geo_point):
    # Check if the value is a string
    if isinstance(geo_point, str):  
        match = re.match(r'POINT \(([-\d.]+) ([-\d.]+)\)', geo_point)
        if match:
            return pd.Series([float(match.group(2)), float(match.group(1))], index=['Latitude', 'Longitude'])
    # Return None if the value is not a valid POINT string
    return pd.Series([None, None], index=['Latitude', 'Longitude'])

# Apply the function to create Latitude and Longitude columns
sdoh_measures_df[['Latitude', 'Longitude']] = sdoh_measures_df['Geolocation'].apply(extract_lat_lng)

# Drop the original Geolocation column
sdoh_measures_df.drop(columns='Geolocation', inplace=True)

In [7]:
sdoh_measures_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28287 entries, 0 to 28286
Data columns (total 12 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   State_Abbr          28287 non-null  object 
 1   State_Desc          28287 non-null  object 
 2   County              28278 non-null  object 
 3   Measure             28287 non-null  object 
 4   Pct_of_Pop          28287 non-null  float64
 5   MOE                 28285 non-null  float64
 6   Total_Population    28287 non-null  int64  
 7   CountyFIPS          28287 non-null  int64  
 8   Measure_ID          28287 non-null  object 
 9   Measure_Short_Name  28287 non-null  object 
 10  Latitude            28278 non-null  float64
 11  Longitude           28278 non-null  float64
dtypes: float64(4), int64(2), object(6)
memory usage: 2.6+ MB


In [8]:
# Calculate and print the unique values in Measure, Measure_ID and Measure_Short_Name
measure_count = sdoh_measures_df['Measure'].nunique()
measure_id_count = sdoh_measures_df['Measure_ID'].nunique()
measure_short_name_count = sdoh_measures_df['Measure_Short_Name'].nunique()
# print("Number of unique Measures: ", measure_count)
# print("Number of unique Measure IDs", measure_id_count)
# print("Number of unique Measure Short Names", measure_short_name_count)

# Create lists of the unique Measures, Measure_IDs and Measure_Short_Names
unique_measures = sdoh_measures_df['Measure'].unique()
unique_measure_ids = sdoh_measures_df['Measure_ID'].unique()
unique_measure_short_names = sdoh_measures_df['Measure_Short_Name'].unique()

measures_key_df = pd.DataFrame({'Measure_ID': unique_measure_ids, 
                                'Measure_Short_Name': unique_measure_short_names, 
                                'Measure': unique_measures
                               })

measures_key_df

,Measure_ID,Measure_Short_Name,Measure
0,HCOST,Housing cost burden,Housing cost burden among households
1,POV150,Poverty,Persons living below 150% of the poverty level
2,CROWD,Crowding,Crowding among housing units
3,AGE65,Aged 65 years or older,Persons aged 65 years or older
4,SNGPNT,Single-parent households,Single-parent households
5,BROAD,No broadband,No broadband internet subscription among house...
6,NOHSDP,No high school diploma,No high school diploma among adults aged 25 ye...
7,UNEMP,Unemployment,Unemployment among people 16 years and older i...
8,REMNRTY,Racial or ethnic minority status,Persons of racial or ethnic minority status


In [20]:
# Export measures_key_df to csv file
measures_key_df.to_csv('../Outputs/sdoh_measures_key.csv', index=False)

In [10]:
sdoh_measures_df['Measure'].value_counts()

Measure
Housing cost burden among households                               3143
Persons living below 150% of the poverty level                     3143
Crowding among housing units                                       3143
Persons aged 65 years or older                                     3143
Single-parent households                                           3143
No broadband internet subscription among households                3143
No high school diploma among adults aged 25 years or older         3143
Unemployment among people 16 years and older in the labor force    3143
Persons of racial or ethnic minority status                        3143
Name: count, dtype: int64

In [11]:
sdoh_measures_df.head()

,State_Abbr,State_Desc,County,Measure,Pct_of_Pop,MOE,Total_Population,CountyFIPS,Measure_ID,Measure_Short_Name,Latitude,Longitude
0,AL,Alabama,Coosa County,Housing cost burden among households,12.8,4.0,10442,1037,HCOST,Housing cost burden,32.931445,-86.243482
1,AL,Alabama,Lamar County,Persons living below 150% of the poverty level,27.4,4.0,13929,1075,POV150,Poverty,33.787085,-88.087431
2,AK,Alaska,Aleutians East Borough,Crowding among housing units,4.6,2.3,3409,2013,CROWD,Crowding,55.245044,-161.997477
3,AR,Arkansas,Dallas County,Crowding among housing units,2.3,2.2,6645,5039,CROWD,Crowding,33.967823,-92.653999
4,AR,Arkansas,Franklin County,Crowding among housing units,2.6,1.8,17159,5047,CROWD,Crowding,35.508558,-93.887665


In [12]:
sdoh_measures_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28287 entries, 0 to 28286
Data columns (total 12 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   State_Abbr          28287 non-null  object 
 1   State_Desc          28287 non-null  object 
 2   County              28278 non-null  object 
 3   Measure             28287 non-null  object 
 4   Pct_of_Pop          28287 non-null  float64
 5   MOE                 28285 non-null  float64
 6   Total_Population    28287 non-null  int64  
 7   CountyFIPS          28287 non-null  int64  
 8   Measure_ID          28287 non-null  object 
 9   Measure_Short_Name  28287 non-null  object 
 10  Latitude            28278 non-null  float64
 11  Longitude           28278 non-null  float64
dtypes: float64(4), int64(2), object(6)
memory usage: 2.6+ MB


In [14]:
# Pivot the DataFrame to make Measure_ID values the column names
sdoh_pivot_df = sdoh_measures_df.pivot_table(
    index = ['State_Abbr', 'State_Desc', 'County', 'CountyFIPS', 'Latitude', 'Longitude', 'Total_Population'], 
    columns = 'Measure_ID',  
    values = 'Pct_of_Pop'
).reset_index()

# Flatten the column names to remove 'Measure_ID' as the index column title
sdoh_pivot_df.columns = [col if not isinstance(col, tuple) else col[1] for col in sdoh_pivot_df.columns]

sdoh_pivot_df.head()

,State_Abbr,State_Desc,County,CountyFIPS,Latitude,Longitude,Total_Population,AGE65,BROAD,CROWD,HCOST,NOHSDP,POV150,REMNRTY,SNGPNT,UNEMP
0,AK,Alaska,Aleutians East Borough,2013,55.245044,-161.997477,3409,11.7,42.5,4.6,16.6,15.3,22.7,87.2,7.9,4.5
1,AK,Alaska,Aleutians West Census Area,2016,51.959447,178.338813,5251,7.0,23.0,8.0,15.5,9.0,11.3,77.1,5.0,3.8
2,AK,Alaska,Anchorage Municipality,2020,61.174250,-149.284329,292545,11.5,7.3,4.5,25.7,5.8,15.1,43.9,6.0,5.4
3,AK,Alaska,Bethel Census Area,2050,60.929141,-160.152625,18514,7.5,25.2,31.7,18.8,18.0,43.9,90.8,8.8,18.1
4,AK,Alaska,Bristol Bay Borough,2060,58.741661,-156.966805,849,13.1,23.5,5.4,7.9,5.3,8.0,58.8,3.8,2.4


In [15]:
sdoh_pivot_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3142 entries, 0 to 3141
Data columns (total 16 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   State_Abbr        3142 non-null   object 
 1   State_Desc        3142 non-null   object 
 2   County            3142 non-null   object 
 3   CountyFIPS        3142 non-null   int64  
 4   Latitude          3142 non-null   float64
 5   Longitude         3142 non-null   float64
 6   Total_Population  3142 non-null   int64  
 7   AGE65             3142 non-null   float64
 8   BROAD             3142 non-null   float64
 9   CROWD             3142 non-null   float64
 10  HCOST             3142 non-null   float64
 11  NOHSDP            3142 non-null   float64
 12  POV150            3142 non-null   float64
 13  REMNRTY           3142 non-null   float64
 14  SNGPNT            3142 non-null   float64
 15  UNEMP             3142 non-null   float64
dtypes: float64(11), int64(2), object(3)
memory

In [16]:
sdoh_pivot_df.dtypes

State_Abbr           object
State_Desc           object
County               object
CountyFIPS            int64
Latitude            float64
Longitude           float64
Total_Population      int64
AGE65               float64
BROAD               float64
CROWD               float64
HCOST               float64
NOHSDP              float64
POV150              float64
REMNRTY             float64
SNGPNT              float64
UNEMP               float64
dtype: object

In [19]:
# Export sdoh_pivot_df to csv file
sdoh_pivot_df.to_csv('../Outputs/sdoh_measures.csv', index=False)

In [21]:
# Create a county FIPS key dataframe 
county_key_df = sdoh_pivot_df[['County', 'State_Abbr', 'State_Desc', 'CountyFIPS', 'Latitude', 'Longitude']]

county_key_df.head()

,County,State_Abbr,State_Desc,CountyFIPS,Latitude,Longitude
0,Aleutians East Borough,AK,Alaska,2013,55.245044,-161.997477
1,Aleutians West Census Area,AK,Alaska,2016,51.959447,178.338813
2,Anchorage Municipality,AK,Alaska,2020,61.174250,-149.284329
3,Bethel Census Area,AK,Alaska,2050,60.929141,-160.152625
4,Bristol Bay Borough,AK,Alaska,2060,58.741661,-156.966805


In [22]:
county_key_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3142 entries, 0 to 3141
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   County      3142 non-null   object 
 1   State_Abbr  3142 non-null   object 
 2   State_Desc  3142 non-null   object 
 3   CountyFIPS  3142 non-null   int64  
 4   Latitude    3142 non-null   float64
 5   Longitude   3142 non-null   float64
dtypes: float64(2), int64(1), object(3)
memory usage: 147.4+ KB


In [23]:
# Export county_key_df to csv file
county_key_df.to_csv('../Outputs/county_key.csv', index=False)